In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from tabulate import tabulate
import numpy as np
import re
import locale
locale.setlocale(locale.LC_ALL, '')
import sys
import subprocess
from scipy.optimize import minimize
from matplotlib import rcParams
from scipy.optimize import basinhopping
rcParams['figure.figsize'] = 9, 9

In [2]:
comps = pd.read_excel('tabulated_data3.xlsm', sheet_name='Compositions')
compNames = list(comps)[1:]
comps

Name McLeod-Mix1 McLeod-Mix2 McLeod-Mix3 McLeod-Mix22  \
0            Group      McLeod      McLeod      McLeod       McLeod   
1           Sample        Mix1        Mix2        Mix3        Mix22   
2               SG    0.705971    0.652476    0.609878     0.698439   
3    Inert Free SG    0.560051    0.558915    0.560063     0.681036   
4              CO2       0.005       0.003      0.0131        0.018   
..             ...         ...         ...         ...          ...   
111            NaN         NaN         NaN         NaN          NaN   
112            NaN         NaN         NaN         NaN          NaN   
113            NaN         NaN         NaN         NaN          NaN   
114            NaN         NaN         NaN         NaN          NaN   
115            NaN         NaN         NaN         NaN          NaN   

    McLeod-Mix24 McLeod-Mix25 Simon&Briggs-10PC Simon&Briggs-20PC API 37-Mix2  \
0         McLeod       McLeod      Simon&Briggs      Simon&Briggs      API 37   
1          Mix24        Mix25              10PC              20PC        Mix2   
2       0.741612      0.59657          0.616048          0.678315    0.789937   
3       0.736766     0.592865           0.55378           0.55378    0.718099   
4         0.0061        0.004                 0                 0      0.0209   
..           ...          ...               ...               ...         ...   
111          NaN          NaN               NaN               NaN         NaN   
112          NaN          NaN               NaN               NaN         NaN   
113          NaN          NaN               NaN               NaN         NaN   
114          NaN          NaN               NaN               NaN         NaN   
115          NaN          NaN               NaN               NaN         NaN   

     ... Custom-A6ST1_3000 Custom-A6ST1_2500 Custom-A6ST1_2000  \
0    ...            Custom            Custom            Custom   
1    ...        A6ST1_3000        A6ST1_2500        A6ST1_2000   
2    ...          1.177425          1.146013          1.129789   
3    ...          0.952983           0.89102          0.851054   
4    ...             0.396            0.4041             0.415   
..   ...               ...               ...               ...   
111  ...               NaN               NaN               NaN   
112  ...               NaN               NaN               NaN   
113  ...               NaN               NaN               NaN   
114  ...               NaN               NaN               NaN   
115  ...               NaN               NaN               NaN   

    Custom-A6ST1_1500 Custom-A6ST1_1000 Custom-A6ST1_500 Custom-D1_4088  \
0              Custom            Custom           Custom         Custom   
1          A6ST1_1500        A6ST1_1000        A6ST1_500        D1_4088   
2            1.132206          1.164653           1.2437       1.463583   
3            0.835685          0.867391          0.98819       1.465396   
4               0.432            0.4552           0.4813        0.01068   
..                ...               ...              ...            ...   
111               NaN               NaN              NaN            NaN   
112               NaN               NaN              NaN            NaN   
113               NaN               NaN              NaN            NaN   
114               NaN               NaN              NaN            NaN   
115               NaN               NaN              NaN            NaN   

    Custom-D1_4089 Custom-D1_4848 Custom-D1_4469  
0           Custom         Custom         Custom  
1          D1_4089        D1_4848        D1_4469  
2         0.949258       0.880221       1.480842  
3         0.942719       0.872138       1.484551  
4          0.01118        0.01157        0.01457  
..             ...            ...            ...  
111            NaN            NaN            NaN  
112            NaN            NaN            NaN  
113            NaN            NaN  

In [5]:
def make_valid_filename(input_string):
    valid_filename = re.sub(r'[^\w.-]', '_', input_string)
    valid_filename = valid_filename.strip()
    valid_filename = re.sub(r'[-_.]{2,}', '_', valid_filename)
    valid_filename = valid_filename.replace('@', '_').replace('#', '_')
    return valid_filename

filename ='HC_BIPS_v8_New_S&K'
phz_template = filename+'.phz' # The .Phz file to use as the base

In [6]:
R, mwAir, degF2R = 10.731577089016, 28.97, 459.67
class mix:
    def __init__(self, name, group, sample, sg, co2, h2s, n2, gas):
        h2 = 0
        self.comps = [co2, h2s, n2, h2, gas]
        self.compnames = ['CO2', 'H2S', 'N2', 'H2', 'Gas']
        self.name = make_valid_filename(name)
        self.group = group
        self.sample = sample
        self.sg = sg
        mws = np.array([44.01, 34.082, 28.014, 2.016, 0])
        self.sg_hc = (sg - (co2 * mws[0] + h2s * mws[1] + n2 * mws[2] + h2*mws[3]) / mwAir) / (1 - co2 - h2s - n2 - h2)
        mw_gas = self.sg_hc * mwAir
        #self.pc = (coefic_pc[0] * mw_gas + coefic_pc[1])/(coefic_pc[2] * mw_gas + coefic_pc[3])
        #self.tc = (coefic_tc[0] * mw_gas + coefic_tc[1] ) /(coefic_tc[2] * mw_gas + coefic_tc[3] )     
        #self.pc = (coefic_pc[0] * mw_gas + coefic_pc[1])
        #self.tc = (coefic_tc[0] * mw_gas + coefic_tc[1] )
        self.expmt = pd.read_excel('tabulated_data3.xlsm', sheet_name=name)
        
    def write_include(self, base_chr): # Write include files for 4 component model
        mws = np.array([44.01, 34.082, 28.014, 2.016, 0])
        df = self.expmt.copy()
        temps = list(set(df['degf'].to_list()))
        temps.sort()
        
        out_str = 'DEFINE MW '+str(self.sg_hc * mwAir)+'\n'
        
        for temp in temps:
            out_str += 'DEFINE degR '+str(temp+459.67)+'\n\n'
            out_str += 'CHARACTERIZATION "CHR_'+self.name+'_'+str(temp)+'" BASE "'+base_chr+'"\n'
            header = ['COMPONENT', 'MW', 'TC,R', 'PC,PSIA', 'AF', 'VTRAN', 'A', 'B']
            rows = []
            for c in range(len(self.compnames)):
                rows.append([self.compnames[c], '', '', '', '', '', '', ''])
            rows[-1][1] = round(self.sg_hc * mwAir, 4)
            #rows[-1][2] = round(self.tc,2)
            #rows[-1][3] = round(self.pc,2)

            out_str += tabulate(rows, header)
            out_str += '\nEND\n\n'
            
            out_str += 'DEFINE degR_Slope CO2_degR_Slope, DEFINE MW_Slope CO2_MW_Slope, DEFINE Intcpt CO2_Intcpt, DEFINE Inert CO2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope H2S_degR_Slope, DEFINE MW_Slope H2S_MW_Slope, DEFINE Intcpt H2S_Intcpt, DEFINE Inert H2S\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope N2_degR_Slope, DEFINE MW_Slope N2_MW_Slope, DEFINE Intcpt N2_Intcpt, DEFINE Inert N2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            out_str += 'DEFINE degR_Slope H2_degR_Slope, DEFINE MW_Slope H2_MW_Slope, DEFINE Intcpt H2_Intcpt, DEFINE Inert H2\n'
            out_str += 'INCLUDE bips_calc.inc\n\n'
            
            ## Inert:Inert BIP Pairs                                                                                                   
            #slopes = [[-0.000879210909090909, -0.00130965, 0.00118182785714286, -0.00150498, 0.00461018571428571, 0.000235450357142857],                                
            #          [0.00009085, 0.000541365245623941, 0.00135899363636364, 0.00103136527385658, 0.00112685210526316, 0.000149872727272727]]                                    
            #intcpts = [[0.239517381818182, -0.085784, 0.203866711904762, 0.85241, 0.400074571428572, 0.204373053571429],                                   
            #           [0.0880919666666667, -0.247960272896669, 0.172206318181818, 0.601450271597967, 0.822955005263158, 0.220908090909091]]                               
            #evals = [np.max, np.max, np.max, np.max, np.min, np.min]                                                                  
            #pairs = ['H2S with CO2', 'CO2 with N2', 'H2S WITH N2', 'CO2 with H2', 'H2S with H2', 'N2 with H2']                       
            #inert_bips = [evals[i]([slopes[0][i]*temp + intcpts[0][i], slopes[1][i]*temp + intcpts[1][i]]) for i in range(len(pairs))]
            #                                                                                                                      
            #for p, pair in enumerate(pairs):                                                                                          
            #    out_str += 'REPLACE BIP of '+pair+' by '+str(inert_bips[p])+'\n'  
                
            #BIP_HC = max(0.000160492*temp + 0.056787492, 1/(0.040933216*temp + 2.454878276)) 
            #BIP_CN = 1/(-0.061940854*temp + -0.494913095)                     
            #BIP_HN = -0.836144851 + (temp * 2.80825315) / (348.2693873 + temp)

            #out_str += 'REPLACE BIP of H2S with CO2 by '+str(BIP_HC)+'\n'
            #out_str += 'REPLACE BIP of CO2 with N2 by '+str(BIP_CN)+'\n'
            #out_str += 'REPLACE BIP of H2S with N2 by '+str(BIP_HN)+'\n\n'
            
            out_str += 'REPLACE BIP of CO2 with H2S by CO2_H2S\n'
            out_str += 'REPLACE BIP of CO2 with N2 by CO2_N2\n'
            out_str += 'REPLACE BIP of CO2 with H2 by CO2_H2\n'
            out_str += 'REPLACE BIP of H2S with N2 by H2S_N2\n'
            out_str += 'REPLACE BIP of H2S with H2 by H2S_H2\n'
            out_str += 'REPLACE BIP of N2 with H2 by N2_H2\n\n'
                
            out_str += '\nINCLUDE '+self.name+'_init.inc\n\n'
            
            out_str += 'MIX '+self.name+'_'+str(temp)+'\n'
            
            rows = []
            for c in range(len(self.compnames)):
                rows.append([self.compnames[c], self.comps[c]])
            out_str += tabulate(rows)
            out_str += '\nEND\n\n'
        
            out_str += 'QIX FEED 1 MOLE '+self.name+'_'+str(temp)+'\n'
            out_str += 'CCE ID "'+self.name+'_'+str(temp)+'"\n'

            exp_df = pd.DataFrame()
        
            filterd_df = df[df['degf'] == temp].copy()
            temps = filterd_df['degf'].to_list()+['Wt']
            press = filterd_df['psia'].to_list()+['']
            zs = filterd_df['Z'].to_list()+[1]
            gss = [1 for z in zs]
            gss[-1] = 0
            
            exp_df['TEMP (F)']= temps
            exp_df['PRES (psia)']= press
            exp_df['Z']= zs
            exp_df['GS']= gss

            out_str += tabulate(exp_df, headers='keys', showindex='never', numalign='left')
            out_str += '\nEND\n\n'

        return out_str


In [7]:
#phz_template = 'Inerts_PR_Match_VSHIFT_Only_AB_CO2_BigLoop_BIPS_Temp_reduced_MW_Slope3_Orig_Omega.phz' # The .Phz file to use as the base


Phz_loc = "C:\\Temp\\PhazeComp\\phazecomp.exe"         # The location of the PhazeComp executable on your system
#Phz_loc = "C:\\Program Files\\PhazeComp\\1.81\\phazecomp.exe"         # The location of the PhazeComp executable on your system

def create_phz(phz_template):
    rms=float("nan")
    try:
        proc = subprocess.run([Phz_loc, phz_template,'NUL',">"], capture_output=True)
    except:
        print("Either phazecomp.exe couldnt be found (is it in "+Phz_loc+" ?), or "+phz_template+" couldn't be found - Exiting..")
        sys.exit()
    
    found_rms = False
    for i in range(len(proc.stderr.splitlines())-1,0,-1):
        line = proc.stderr.splitlines()[i].decode("utf-8")
        if "% RMS average weighted" in line:
            rms = float(line.split()[2])
            found_rms = True
            break
    if not found_rms:
        print("Error: No RMS for ",phz_template)
    return rms

In [8]:
mw_hcs = [16.043, 25.977, 33.864, 45.08779652]                                                                                                                                    
comp_dic = {}                                                                                     
#print(x)                                                                                         
for c in range(len(compNames)):                                                                   
    name = compNames[c]                                                                           
    group = comps.iloc[0, c+1]                                                                    
    sample = comps.iloc[1, c+1]                                                                   
    sg = comps.iloc[2, c+1]                                                                       
    co2 = comps.iloc[4, c+1]                                                                      
    h2s = comps.iloc[5, c+1]                                                                      
    n2 = comps.iloc[6, c+1]                                                                       
    gas = 1 - co2 - h2s - n2                                                                      
    comp_dic[name] = mix(name, group, sample, sg, co2, h2s, n2, gas)        
                                                                                                  
# 4 component include files                                                                       
for c in range(len(compNames)):                                                                   
    file_path = make_valid_filename(compNames[c])                                                 
    string = comp_dic[compNames[c]].write_include('RealGas')                                      
    with open('.\\'+filename+'\\'+file_path+'.inc', 'w') as file:                                 
        print('.\\'+filename+'\\'+file_path+'.inc')                                               
        file.write(string)                                                                        
                                                                                                  
#gergs = ['lean', 'leanmid', 'mid', 'rich']                                                       
#gerg_mws = [16.043, 25.977, 33.864, 45.08779652]                                                 
                                                                                                  
#for g, gerg in enumerate(gergs):                                                                 
#    out_str = 'DEFINE MW '+str(mw_hcs[g])+'\n'                                                   
#    out_str += 'DEFINE mult_tc '+str(1/(coefic_tc[0] * gerg_mws[g] + coefic_tc[1]) )+'\n'        
#    out_str += 'DEFINE mult_pc '+str(1/(coefic_pc[0] * gerg_mws[g] + coefic_pc[1]))+'\n'         
#    with open('.\\GERG\\'+gerg+'_init.inc', 'w') as file:                                        
#        file.write(out_str)                                                                      
                                                                                                  
#rms = create_phz(phz_template)                                                                   
#print(x, rms)                                                                                    
#return rms                                                                                       

.\HC_BIPS_v8_New_S&K\McLeod-Mix1.inc
.\HC_BIPS_v8_New_S&K\McLeod-Mix2.inc
.\HC_BIPS_v8_New_S&K\McLeod-Mix3.inc
.\HC_BIPS_v8_New_S&K\McLeod-Mix22.inc
.\HC_BIPS_v8_New_S&K\McLeod-Mix24.inc
.\HC_BIPS_v8_New_S&K\McLeod-Mix25.inc
.\HC_BIPS_v8_New_S&K\Simon_Briggs-10PC.inc
.\HC_BIPS_v8_New_S&K\Simon_Briggs-20PC.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix2.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix17.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix25.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix26.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix402.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix601.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix705.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix707.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix722.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix727.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix729.inc
.\HC_BIPS_v8_New_S&K\API_37-Mix735.inc
.\HC_BIPS_v8_New_S&K\Satter_Campbell-MixA.inc
.\HC_BIPS_v8_New_S&K\Satter_Campbell-MixB.inc
.\HC_BIPS_v8_New_S&K\Satter_Campbell-MixC.inc
.\HC_BIPS_v8_New_S&K\Satter_Campbell-MixD.inc
.\HC_BIPS_v8_New_S&K\Buxton_Campbell-Mix1.

In [9]:
def fn2(x):
    x = denorm_solx(x, bounds)
    coefic_pc = x[:2]
    coefic_tc = x[2:]
    
    # 4 component include files
    for comp in compNames:
        file_path = make_valid_filename(comp)
        mw = comp_dic[comp].sg_hc * mwAir
        
        mw_ch4, tc_ch4, pc_ch4 = 16.043, 343.008, 667.029
        pc_int = pc_ch4 * (coefic_pc[1] * mw_ch4 + 1) - coefic_pc[0] * mw_ch4
        tc_int = tc_ch4 * (coefic_tc[1] * mw_ch4 + 1) - coefic_tc[0] * mw_ch4
        
        pc = (coefic_pc[0]*mw + pc_int)/(coefic_pc[1]*mw + 1)
        tc = (coefic_tc[0]*mw + tc_int)/(coefic_tc[1]*mw + 1)

        #out_str += '\nREPLACE TC of Gas by ?MW? R\n'
            #out_str += 'MULTIPLY TC of Gas by TC_slope\n'
            #out_str += 'INCREASE TC of Gas by TC_int R\n'
            #out_str += 'MULTIPLY TC of Gas by ?mult_tc?\n\n' 
            
            #out_str += 'REPLACE PC of Gas by ?MW? psia\n'
            
        out_str = 'REPLACE TC of Gas by '+str(tc)+' R\n'
        out_str += 'REPLACE PC of Gas by '+str(pc)+' psia\n'
        
        with open('.\\'+filename+'\\'+file_path+'_init.inc', 'w') as file:
            #print('.\\'+filename+'\\'+file_path+'_init.inc')
            file.write(out_str)
    
    gergs = ['lean', 'leanmid', 'mid', 'rich']
    gerg_mws = [16.043, 25.977, 33.864, 45.08779652]

    for g, gerg in enumerate(gergs):
        
        pc_int = pc_ch4 * (coefic_pc[1] * mw_ch4 + 1) - coefic_pc[0] * mw_ch4
        tc_int = tc_ch4 * (coefic_tc[1] * mw_ch4 + 1) - coefic_tc[0] * mw_ch4
        
        pc = (coefic_pc[0]*gerg_mws[g] + pc_int)/(coefic_pc[1]*gerg_mws[g] + 1)
        tc = (coefic_tc[0]*gerg_mws[g] + tc_int)/(coefic_tc[1]*gerg_mws[g] + 1)

        out_str = 'REPLACE TC of Gas by '+str(tc)+' R\n'
        out_str += 'REPLACE PC of Gas by '+str(pc)+' psia\n'
        with open('.\\'+filename+'_GERG\\'+gerg+'_gas_init.inc', 'w') as file:
            file.write(out_str)
        
    rms = create_phz(phz_template)
    print(x, rms)
    return rms

In [11]:
fn2(x)

[0.000746468, 1.313497747, 0.001116319, 0.353188266] 0.9104


0.9104

In [11]:
def norm_x0(x, bounds):
    x0 = []
    for i, bound in enumerate(bounds):
        span = bound[1] - bound[0]
        x0.append((x[i] - bound[0])/span)
    return x0

def denorm_solx(x, bounds):
    x0 = []
    for i, bound in enumerate(bounds):
        span = bound[1] - bound[0]
        x0.append(x[i]*span + bound[0])
    return x0
    

In [27]:
x0 = [-12.627314595959447, -0.018690583803656374, 9.04430052809979, -0.0028206990940295718]
fn2(norm_x0(x0, bounds))

[-12.627314595959447, -0.018690583803656374, 9.04430052809979, -0.0028206990940295718] 1.097


1.097

In [14]:
#x0 = [-0.004289489, -4.521919227, 756.8, -0.08817282, 12.06420435, 169.2] # Base Sutton
# x0 = [-3.55897620e-03, -2.78434498e+00,  7.04262503e+02, -1.00841444e-01, 1.17914323e+01,  1.80343787e+02] # v3 previous best

#x0 = [-9.38491450e-01,  9.13571165e+02,  4.90854197e-03,  1.29146359e+00,  6.24106819e+00,  9.74288258e+00,  7.23867099e-03,  2.08134634e-01]
#x0 = [-2.23048,6.87323e+02,0,1,8.21344,2.12017e+02,0,1]
#x0 = [0.00529424, 1.59103386, 0.00234167, 0.31144261]
#x0 = [7.05821902e-04, 1.70270381e+00, 1.19687273e-03, 3.35493340e-01]

#x0 = [5.76161228e-03, 1.92000000e+00, -1.89318001e-04, 4.69900642e-01]
#bounds = [(min(x*0.5, x*1.5), max(x*0.5, x*1.5)) for x in x0]
#bounds = [(-3.3875000000000002, -0.4625), (400, 1375.5), (-0.07, 0.07), (0.44, 1.92),
# (3.110137083, 9.330411249), (4.7625179755, 300), (-0.07, 0.07), (0.1035, 1.5)]

#x0 = [7.39469863e-04, 1.42432412e+00, 1.21050842e-03, 3.33321426e-01]
#x0 = [0.007020498750000034, 1.5010786600000001, 0.0024482299999999957, 0.3219414605] # 1.269 first go
#x0 = [0.00530666, 1.55252724, 0.00232565, 0.31546929] # From best fit with inerts folder (v6)
#x0 = [0.005337679293082267, 1.551293419112061, 0.0023166104602457584, 0.3248137521706216] # Current best match before diagnosing issue

#x0 = [0.000686956691, 1.69866602, 0.00126240593, 0.333600083] # From v6
#x0 = [0.0007168250962455172, 1.7634944280537534, 0.0014088764571192403, 0.27202818055460637]

x0 = [-1.97, 0.000460678, 12.8518, 0.005287488]
x0 = [-1.9885028991699203, 0.0004670255377502439, 12.25894699707031, 0.005765036850585936] #1.408
x0 = [-1.523754301, 0.00341463, 13.81093753, 0.007287103] # 1.037 Initial

x0 = [-12.73791638, -0.017639786, 9.007198296, -0.000183249622066317] # 1.037 Initial
x0 = [-12.733552535559795, -0.017618840594713248, 9.048069504043694, -0.001704442068847692] # 1.035 initial (without rich)
x0 = [-12.607780250106046, -0.018590048732292447, 9.0626816494811, -0.0025542092360408136]
bounds = [(-15, 0.5), (-0.20, 0.05), (8, 30), (-0.1, 0.05)]
x = norm_x0(x0, bounds)

sol = minimize(fn2, x, method='Nelder-Mead', bounds=[(0, 1), (0, 1), (0, 1), (0, 1)])
# 1.184 with v6 + H2 BIP's. Now trying with constant BIP's

[-12.607780250106046, -0.018590048732292447, 9.0626816494811, -0.0025542092360408136] 1.092
[-12.488169262611349, -0.018590048732292447, 9.0626816494811, -0.0025542092360408136] 1.158
[-12.607780250106046, -0.00951955116890707, 9.0626816494811, -0.0025542092360408136] 10.52
[-12.607780250106046, -0.018590048732292447, 9.115815731955154, -0.0025542092360408136] 1.093
[-12.607780250106046, -0.018590048732292447, 9.0626816494811, 0.0023180803021571467] 2.478
[-12.547974756358698, -0.027660546295677824, 9.089248690718128, -0.00011806446694183348] 8.669
[-12.562926129795535, -0.02312529751398512, 9.08260693040887, -0.0007271006592165924] 4.089
[-12.59282887666921, -0.014054799950599772, 9.069323409790357, -0.0019451730437660686] 6.473
[-12.570401816513954, -0.020857673123138798, 9.079286050254241, -0.001031618755353958] 7.116
[-12.607780250106046, -0.018590048732292447, 9.089248690718128, -0.0025542092360408136] 1.093
[-12.547974756358698, -0.018590048732292447, 9.0626816494811, -0.00255420

In [15]:
# Run once at the end to ensure the directory of experiments reflects the solution case
fn2(sol.x)

[-12.607780250106046, -0.018590048732292447, 9.0626816494811, -0.0025542092360408136] 1.092


1.092